In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

df = pd.read_csv("train_data.csv")
test = pd.read_csv("test_data.csv")

result = []

result.append((1, 1, df["ID"].size))
result.append((3, 1, df["Month"].unique().size))


v = df["SSN"].values
w = []
for i in v:
    if i.endswith("20"):
        w.append(i)

result.append((4, 1, len(list(set(w)))))

df.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x6762,CUS_0x6fe2,January,Schnurra,25,164-17-6558,Accountant,62214.36,4997.530000,9,...,_,2892.91,31.737712,13 Years and 10 Months,Yes,339.971582,216.583807190171,High_spent_Small_value_payments,203.19761100877272,-1
1,0x18b64,CUS_0xb13c,July,Jennifert,33,528-06-6283,_______,14613.65,1334.804167,8,...,Bad,2882.07,27.414562,2 Years and 3 Months,Yes,67.975819,57.54453501725725,High_spent_Small_value_payments,267.96006258969163,0
2,0xb8d5,CUS_0xa0c3,August,Chanf,56,355-23-1301,Lawyer,101112.6,8600.050000,8,...,Good,399.49,36.516160,29 Years and 3 Months,No,143.856735,132.07082261639974,High_spent_Large_value_payments,824.0774428659337,1
3,0x2ec3,CUS_0x6732,February,alazsh,50,060-69-0738,Journalist,140390.32,11888.193333,5,...,Good,1423.23,36.955252,30 Years and 9 Months,No,182.160424,133.2130344228415,High_spent_Medium_value_payments,1123.4458751336915,1
4,0x1b36,CUS_0x8c69,January,Toonkelf,24,070-07-9414,Musician,20188.02,1851.335000,4,...,_,399.92,28.542585,20 Years and 2 Months,No,25.580281,147.63621347336863,Low_spent_Large_value_payments,281.91700571081503,0


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44207 entries, 0 to 44206
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        44207 non-null  object 
 1   Customer_ID               44207 non-null  object 
 2   Month                     44207 non-null  object 
 3   Name                      44207 non-null  object 
 4   Age                       44207 non-null  object 
 5   SSN                       44207 non-null  object 
 6   Occupation                44207 non-null  object 
 7   Annual_Income             44207 non-null  object 
 8   Monthly_Inhand_Salary     44207 non-null  float64
 9   Num_Bank_Accounts         44207 non-null  int64  
 10  Num_Credit_Card           44207 non-null  int64  
 11  Interest_Rate             44207 non-null  int64  
 12  Num_of_Loan               44207 non-null  object 
 13  Type_of_Loan              44207 non-null  object 
 14  Delay_

In [3]:
def Remove_(str):
    spl = str.split("_")
    for x in spl:
        if (x != ''):
            return x


def ProcessDate(date):
    date = Remove_(date)
    years = date.split(' ')[0]
    months = date.split(' ')[3]
    return int(years)*12+int(months)


def StringToFloat(str):
    str = Remove_(str)
    if (str is None):
        return -1
    return float(str)


def StringToInt(str):
    str = Remove_(str)
    return int(str)


def Age(age):
    age = Remove_(age)
    age = int(age)
    if (age > 90):
        age = -1
    elif (age < 18):
        age = -1
    return age

In [4]:
def Modify(df):
    # Remove empty
    # df = df.drop(df[df["Occupation"] == "_______"].index, inplace=False)
    # df["Occupation"].value_counts()

    # Drop useless info
    df = df.drop(columns=["Name", "SSN", "Customer_ID", "Month"])
    
    df = df.dropna()

    # TODO: Mabye change this:
    df = df.drop(columns=[
        "Type_of_Loan",
        "Amount_invested_monthly",
        #"Occupation",
        #"Payment_Behaviour",
    ])

    # Process 
    df = pd.get_dummies(df, columns=["Occupation"])
    df = pd.get_dummies(df, columns=["Payment_Behaviour"])
    
    df["Credit_Mix"] = df["Credit_Mix"].map(arg={"Standard": 1, "Good": 2, "Bad": 0, "_": -1})
    df["Payment_of_Min_Amount"] = df["Payment_of_Min_Amount"].map(arg={"Yes": 1, "No": 0, "NM": -1})
    
    df["Credit_History_Age"] = df["Credit_History_Age"].map(arg=ProcessDate)
    df["Age"] = df["Age"].map(arg=Age)
    
    df["Monthly_Balance"] = df["Monthly_Balance"].replace("__-333333333333333333333333333__", "0")
    df["Monthly_Balance"] = df["Monthly_Balance"].map(arg=StringToFloat)
    
    df["Annual_Income"] = df["Annual_Income"].map(arg=StringToFloat)
    df["Num_of_Loan"] = df["Num_of_Loan"].map(arg=StringToFloat)
    df["Num_of_Delayed_Payment"] = df["Num_of_Delayed_Payment"].map(arg=StringToFloat)
    df["Changed_Credit_Limit"] = df["Changed_Credit_Limit"].map(arg=StringToFloat)
    df["Outstanding_Debt"] = df["Outstanding_Debt"].map(arg=StringToFloat)

    return df

In [5]:
df = Modify(df)
test = Modify(test)

df.head()

,ID,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,...,Occupation_Teacher,Occupation_Writer,Occupation________,Payment_Behaviour_!@9#%8,Payment_Behaviour_High_spent_Large_value_payments,Payment_Behaviour_High_spent_Medium_value_payments,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments
0,0x6762,25,62214.36,4997.530000,9,7,19,9.0,58,20.0,...,False,False,False,False,False,False,True,False,False,False
1,0x18b64,33,14613.65,1334.804167,8,583,22,6.0,20,16.0,...,False,False,True,False,False,False,True,False,False,False
2,0xb8d5,56,101112.60,8600.050000,8,6,4,2.0,18,20.0,...,False,False,False,False,True,False,False,False,False,False
3,0x2ec3,50,140390.32,11888.193333,5,2,4,3.0,3,0.0,...,False,False,False,False,False,True,False,False,False,False
4,0x1b36,24,20188.02,1851.335000,4,3,6,3.0,10,5.0,...,False,False,False,False,False,False,False,True,False,False


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44207 entries, 0 to 44206
Data columns (total 43 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   ID                                                  44207 non-null  object 
 1   Age                                                 44207 non-null  int64  
 2   Annual_Income                                       44207 non-null  float64
 3   Monthly_Inhand_Salary                               44207 non-null  float64
 4   Num_Bank_Accounts                                   44207 non-null  int64  
 5   Num_Credit_Card                                     44207 non-null  int64  
 6   Interest_Rate                                       44207 non-null  int64  
 7   Num_of_Loan                                         44207 non-null  float64
 8   Delay_from_due_date                                 44207 non-null  int64  


In [7]:
def TestAcc(df):
    X = df
    X = X.drop(columns=["ID", "Credit_Score"])
    y = df["Credit_Score"]
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)
    ml = RandomForestClassifier()
    ml.fit(X_train, Y_train)
    acc = ml.score(X_test, Y_test)
    return acc

In [8]:
TestAcc(df)

0.7756163763854331

In [9]:
X = df
X = X.drop(columns=["ID", "Credit_Score"])
y = df["Credit_Score"]
ml = RandomForestClassifier()
ml.fit(X, y)

Z = test
Z = Z.drop(columns=["ID"])
ans = ml.predict(Z)

In [10]:
idx = 0


sum = 0
idx = 0
total = 0
for i in df["Credit_Utilization_Ratio"]:
    if (i >= 25):
        sum += df["Monthly_Inhand_Salary"][idx]
        total += 1
    idx += 1

result.append((2, 1, int(sum/total)))

idx = 0
for i in test["ID"]:
    result.append((5, i, ans[idx]))
    idx += 1

df_out = pd.DataFrame(result, columns=["subtaskID", "datapointID", "answer"])
df_out.to_csv("output.csv", index=False)